<a href="https://colab.research.google.com/github/ravellys/Bootcamp-IGTI/blob/main/desafio_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio - Módulo 5 - IGTI
---
## Ciência de Dados com Python
## Bootcamp Desenvolvedor Python
## Lucas Ravellys

Observaçoes:
O dataset utilizado possui as seguintes colunas:
0. Número de vezes em que ficou grávida.
1. Concentração de glicose.
2. Pressão diastólica (mm Hg).
3. Espessura da dobra cutânea do tríceps (mm).
4. Insulina (mu U/ml).
5. Índice de massa corporal (peso em kg/(altura em m)^2).
6. Histórico familiar de diabetes.
7. Idade (anos).
8. Classificação (0 ou 1 - 0 não diabético / 1 diabético ).

In [19]:
import joblib
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

def pre_tratamento_dados(df, feature_saida):
  # Entrada / Saída
  x = df.drop(columns=['CLASSIFICACAO']) #entrada
  y = df['CLASSIFICACAO'] # saida

  # Normalização dos dados
  from sklearn.preprocessing  import MinMaxScaler
  normaliza = MinMaxScaler() #objeto para a normalização
  entradas_normalizadas=normaliza.fit_transform(x)
  return entradas_normalizadas, y, normaliza


def modelo_ml(model, x, y):
  # treino / teste
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=42)

  clf = model
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  from sklearn.metrics import accuracy_score
  acc= accuracy_score(y_test, y_pred)

  return clf, acc

def selecao_melhor_modelo(modelos, df, feature_saida):
  x, y, normal = pre_tratamento_dados(df, feature_saida)

  lista_modelos = list(modelos.keys())
  melhor = modelo_ml(modelos[lista_modelos[0]], x=x, y=y)

  for modelo_name in lista_modelos[1:]:
    modelo = modelo_ml(modelos[modelo_name], x=x, y=y)
    if modelo[1] > melhor[1]:
      melhor = modelo

  return melhor, normal     


modelos = dict(
    KNeighborsClassifier = KNeighborsClassifier(n_neighbors=5),
    MLPClassifier = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 10), random_state=1),
    DecisionTreeClassifier = DecisionTreeClassifier(random_state=1)
)

if __name__ == '__main__':

  df = pd.read_csv('https://raw.githubusercontent.com/ravellys/Diabetes_predict/main/dataset/pima-indians-diabetes.csv', header=None)
  df.columns = ['NUM_GRAV', 'CONCENTRACAO_GLICOSE', 'PRESSSAO_DIASTOLICA', 'ESPESSURA_TRICEPS', 'INSULINA', 'IMC', 'HISTORICO_FAMILIAR', 'IDADE', 'CLASSIFICACAO']

  melhor_modelo = selecao_melhor_modelo(modelos, df=df, feature_saida='CLASSIFICACAO')
  nome_arquivo = 'melhor_modelo_ml.sav'
  joblib.dump(melhor_modelo[0][0], nome_arquivo)
  nome_arquivo = 'data_normaliza.sav'
  joblib.dump(melhor_modelo[1], nome_arquivo)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [38]:
# carregando modelo
modelo_salvo = joblib.load('/content/melhor_modelo_ml.sav')
normaliza = joblib.load('/content/data_normaliza.sav')

# usando o modelo
df = pd.read_csv('https://raw.githubusercontent.com/ravellys/Diabetes_predict/main/dataset/pima-indians-diabetes.csv', header=None)
df.columns = ['NUM_GRAV', 'CONCENTRACAO_GLICOSE', 'PRESSSAO_DIASTOLICA', 'ESPESSURA_TRICEPS', 'INSULINA', 'IMC', 'HISTORICO_FAMILIAR', 'IDADE', 'CLASSIFICACAO']
x_ = df.drop(columns=['CLASSIFICACAO']) #entrada

x_norm = normaliza.transform(x_)

print(
f"""
entrada: {x_norm[0]}
saída: {modelo_salvo.predict([x_norm[0]])}
""")



entrada: [0.35294118 0.74371859 0.59016393 0.35353535 0.         0.50074516
 0.23441503 0.48333333]
saída: [1]

